In [75]:
import pandas as pd 
import quandl
from pprint import pprint
from datetime import datetime
from collections import defaultdict
quandl.read_key()

In [8]:
sparse_tickers_df = pd.read_csv('historical_stock_mentions.csv')
sparse_tickers_df = sparse_tickers_df.rename(columns={'Unnamed: 0': 'Date'})
sparse_tickers_df = sparse_tickers_df.set_index('Date')
sparse_tickers_df.head()

,AA,AABA,AAC,AAL,AAMC,AAME,AAN,AAOI,AAON,AAP,...,ZTEST,ZTO,ZTR,ZTS,ZUMZ,ZUO,ZVO,ZYME,ZYNE,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-04-15,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-04-14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-04-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-04-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
condensed_df = sparse_tickers_df.loc[:, (sparse_tickers_df != 0).any(axis=0)]
condensed_df.head()

,AA,AAC,AAL,AAN,AAOI,AAP,AAPL,AAU,AAWW,AAXJ,...,ZNGA,ZOM,ZS,ZSAN,ZTO,ZUMZ,ZUO,ZYME,ZYNE,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-04-15,2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2021-04-14,0,0,0,0,0,0,3,0,0,0,...,0,6,0,0,0,0,0,0,0,0
2021-04-09,0,0,0,0,0,0,3,0,0,0,...,0,2,0,0,0,0,0,0,0,0
2021-04-08,0,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
nltk_stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
nltk_stopwords = nltk_stopwords + ['ON', 'DO', 'FOR', 'ARE', 'ALL', 'OR', 
                                   'SO', 'OUT', 'ONE', 'BIG', 'NEW', 'LOVE', 
                                   'BY', 'TOO', 'AI', 'AN', 'TD', 'NEXT', 'SEE', 'GO']
                                    
nltk_stopwords = set([s.upper() for s in nltk_stopwords])

to_drop = [column for column in condensed_df.columns if column in nltk_stopwords]

condensed_df = condensed_df.drop(columns=to_drop)
s = condensed_df.sum()

In [83]:
top_df = condensed_df[s.sort_values(ascending=False).index[:20]]
top_df.head()

,GME,AMC,NOK,BB,TSLA,SNDL,NIO,NAKD,AMD,AAPL,TLRY,AAL,MSFT,AMZN,APHA,PRPL,CTRM,BABA,BBBY,FB
Date,,,,,,,,,,,,,,,,,,,,
2021-04-16,174,91,1,4,3,12,6,1,0,0,3,0,0,1,7,0,1,5,0,0
2021-04-15,163,98,7,10,6,10,9,1,1,0,15,0,0,0,13,0,0,1,1,0
2021-04-14,157,40,1,5,10,11,5,1,4,3,7,0,1,2,5,0,4,0,1,0
2021-04-09,172,42,3,22,1,4,1,4,0,3,2,0,1,0,2,0,0,0,0,0
2021-04-08,196,61,5,12,3,12,3,9,2,4,4,0,0,0,4,0,2,2,1,1


In [90]:
def inner_dict():
    return {}


def nested_dict():
    return defaultdict(inner_dict)

open_data = nested_dict()
close_data = nested_dict()
high_data = nested_dict()
low_data = nested_dict()
volume_data = nested_dict()

for ticker in top_df.columns:
    print(f"Getting Price Data for: {ticker}")
    this_share = share.Share(ticker)
    ticker_data = None
    
    try:
        ticker_data = this_share.get_historical(share.PERIOD_TYPE_YEAR, 2,
                                                share.FREQUENCY_TYPE_DAY, 1)
    except YahooFinanceError as e:
        print(e.message)
    
    # FIXME: Inefficient AF but it's late and I'm tired
    for i, timestamp in enumerate(ticker_data['timestamp']):
        date = datetime.fromtimestamp(timestamp / 1e3).strftime('%m-%d-%y')
        open_data[date][ticker] = ticker_data['open'][i]
        close_data[date][ticker] = ticker_data['close'][i]
        high_data[date][ticker] = ticker_data['high'][i]
        low_data[date][ticker] = ticker_data['low'][i]
        volume_data[date][ticker] = ticker_data['volume'][i]
        

Getting Price Data for: GME
Getting Price Data for: AMC
Getting Price Data for: NOK
Getting Price Data for: BB
Getting Price Data for: TSLA
Getting Price Data for: SNDL
Getting Price Data for: NIO
Getting Price Data for: NAKD
Getting Price Data for: AMD
Getting Price Data for: AAPL
Getting Price Data for: TLRY
Getting Price Data for: AAL
Getting Price Data for: MSFT
Getting Price Data for: AMZN
Getting Price Data for: APHA
Getting Price Data for: PRPL
Getting Price Data for: CTRM
Getting Price Data for: BABA
Getting Price Data for: BBBY
Getting Price Data for: FB


In [89]:
open_df = pd.DataFrame.from_dict(open_data, orient='index')
open_df.tail()

,GME,AMC,NOK,BB,TSLA,NIO,NAKD,AMD,AAPL,TLRY,AAL,MSFT,AMZN,APHA,PRPL,CTRM,BABA,BBBY,FB,SNDL
04-13-21,141.880005,8.66,4.08,9.07,712.700012,37.480000,0.5717,79.669998,132.440002,16.389999,22.180000,257.260010,3400.850098,13.230,34.340000,0.442,243.660004,29.26,312.209991,0.912
04-14-21,143.570007,9.00,4.13,9.05,770.700012,39.230000,0.5639,79.879997,134.940002,18.090000,22.950001,257.480011,3404.040039,14.720,33.759998,0.438,244.369995,24.07,307.299988,0.961
04-15-21,163.000000,9.88,4.12,9.10,743.099976,37.020000,0.5700,80.320000,133.820007,17.680000,22.650000,257.929993,3371.000000,14.250,33.930000,0.437,240.000000,24.58,306.339996,0.908
04-16-21,156.000000,10.00,4.15,8.69,728.650024,34.689999,0.4985,83.300003,134.300003,16.760000,22.059999,259.470001,3380.000000,13.644,33.500000,0.476,241.889999,24.18,308.170013,0.823
04-19-21,171.800003,9.48,4.17,8.65,719.599976,36.779999,0.5029,82.129997,133.509995,17.260000,21.969999,260.190002,3390.330078,14.160,34.939999,0.457,237.440002,25.40,305.010010,0.820


In [88]:
close_df = pd.DataFrame.from_dict(close_data, orient='index')
close_df.tail()

,GME,AMC,NOK,BB,TSLA,NIO,NAKD,AMD,AAPL,TLRY,AAL,MSFT,AMZN,APHA,PRPL,CTRM,BABA,BBBY,FB,SNDL
04-13-21,140.990005,8.84,4.13,9.10,762.320007,38.480000,0.5663,80.190002,134.429993,18.320000,22.559999,258.489990,3400.000000,14.85,33.910000,0.448,241.889999,27.930000,309.760010,0.980
04-14-21,166.529999,9.35,4.13,9.00,732.229980,37.020000,0.5550,78.550003,132.029999,17.400000,22.370001,255.589996,3333.000000,14.23,33.500000,0.428,239.229996,24.520000,302.820007,0.900
04-15-21,156.440002,9.90,4.13,8.77,738.849976,35.660000,0.5150,83.010002,134.500000,16.940001,22.129999,259.500000,3379.090088,13.59,33.349998,0.455,239.089996,24.540001,307.820007,0.852
04-16-21,154.690002,9.33,4.18,8.70,739.780029,36.090000,0.4778,82.150002,134.160004,17.200001,22.030001,260.739990,3399.439941,14.14,35.540001,0.451,238.690002,25.520000,306.179993,0.847
04-19-21,164.369995,9.66,4.17,8.62,714.630005,36.779999,0.5560,81.110001,134.839996,17.440001,21.549999,258.739990,3372.010010,14.41,34.630001,0.488,234.779999,25.090000,302.239990,0.940
